In [1]:
from typing import Annotated, Sequence, TypedDict
from dotenv import load_dotenv  
from langchain_core.messages import BaseMessage # The foundational class for all message types in LangGraph
from langchain_core.messages import ToolMessage # Passes data back to LLM after it calls a tool such as the content and the tool_call_id
from langchain_core.messages import SystemMessage,HumanMessage # Message for providing instructions to the LLM
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode
import os
import requests
from bs4 import BeautifulSoup
load_dotenv()

True

In [2]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]


In [3]:
@tool
def get_news():
    """
    This tool scrapes top headlines and detailed news content related to the stock market, companies, economy, and business
    published today on Moneycontrol.com. It is designed to provide real-time news data for downstream summarisation and analysis 
    agents in a financial trading system.
    """

    url = "https://www.moneycontrol.com/news/business/stocks/"
    response = requests.get(url)

    soup = BeautifulSoup(response.content, "lxml")
    # print(soup.prettify())
    tags=soup.find_all('p')
    news=[]
    for tag in tags :
        if (str(type(tag.string)) == "<class 'bs4.element.NavigableString'>"
                and len(tag.string) > 35):
            news.append(tag.string)
            news.append("###")
    return str(news)

In [4]:
tools = [get_news]

model = ChatOpenAI(model = "gpt-4o").bind_tools(tools)

In [5]:
def stock_summarizer(state:AgentState) -> AgentState:
    system_prompt = SystemMessage(content=
        "You are a financial news analyst helping a stock trading system."
    )
    response = model.invoke([system_prompt] + state["messages"])
    return {"messages": [response]}

In [6]:
def should_continue(state: AgentState): 
    messages = state["messages"]
    last_message = messages[-1]
    if not last_message.tool_calls: 
        return "end"
    else:
        return "continue"

In [7]:
graph = StateGraph(AgentState)
graph.add_node("our_agent", stock_summarizer)


tool_node = ToolNode(tools=tools)
graph.add_node("tools", tool_node)

graph.set_entry_point("our_agent")

graph.add_conditional_edges(
    "our_agent",
    should_continue,
    {
        "continue": "tools",
        "end": END,
    },
)

graph.add_edge("tools", "our_agent")

app = graph.compile()

In [8]:
prompts="""
Your task is to summarize the following news article(s) in a **clear, concise, and fact-preserving** way. Focus only on the essential information that would be relevant for financial markets, investors, or traders.
DO:
- Retain key facts, numbers, dates, company names, and events
- Write in a neutral tone (no opinions or interpretation)
- Omit irrelevant or repetitive information
- Avoid any kind of analysis, judgment, or sentiment
- Use full sentences in a formal news summary style
DO NOT:
- Perform sentiment analysis
- Classify or extract entities
- Infer or hallucinate missing details
"""

In [9]:
inputs = {"messages": [(f"user,{prompts}")]}


In [10]:
response=app.invoke(inputs)

In [12]:
for msg in response["messages"]:
    print(msg.pretty_print())

================================ Human Message =================================

user,
Your task is to summarize the following news article(s) in a **clear, concise, and fact-preserving** way. Focus only on the essential information that would be relevant for financial markets, investors, or traders.
DO:
- Retain key facts, numbers, dates, company names, and events
- Write in a neutral tone (no opinions or interpretation)
- Omit irrelevant or repetitive information
- Avoid any kind of analysis, judgment, or sentiment
- Use full sentences in a formal news summary style
DO NOT:
- Perform sentiment analysis
- Classify or extract entities
- Infer or hallucinate missing details

None
================================== Ai Message ==================================
Tool Calls:
  get_news (call_yRt8ojOMZk6wpkiet0VQf6vi)
 Call ID: call_yRt8ojOMZk6wpkiet0VQf6vi
  Args:
None
================================= Tool Message =================================
Name: get_news

["See the top gainers, lo